# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.columns


Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
#calculate the school count by unique and sum .
#school_data_complete["school_name"].unique()
School_count=len(school_data_complete['school_name'].unique())
#print(School_count)

#calculate the student count and budget.
student_count=school_data_complete["student_name"].count()
#print(student_count)
total =school_data["budget"].sum()
#print(total)

#calculate overall passing rate & average of math and reading score
math_average =school_data_complete["math_score"].mean()
#print(math_average)
reading_average =school_data_complete["reading_score"].mean()
#print(reading_average)
overall_score=(math_average+reading_average)/2
#print(overall_score)
math_above_70=school_data_complete.loc[school_data_complete["math_score"]>=70]
count_math_above_70=len(math_above_70)
#print(total_stud_above_70)
per_stu_math= (count_math_above_70/student_count)*100
#print(per_stu_math)
read_above_70=school_data_complete.loc[school_data_complete["reading_score"]>=70]
count_read_above_70=len(read_above_70)
#print(total_stud_above_70_read)
per_stu_read= (count_read_above_70/student_count)*100
#print(per_stu_read)

df=pd.DataFrame({"Total Schools": School_count,
                "Total Students":student_count,
                "Total Budget" : [total],
                 "Average Math Score": [math_average],
                 "Average Reading Score" : [reading_average],
                 "% Passing Math" : [per_stu_math],
                 "% Passing Reading" :[per_stu_read],
                 "% Overall Passing Rate" : [overall_score]})
df

#df = df[["Total Schools","Total Students","Total Budget","Average Math Score", "Average Reading Score",
           # "% Passing Math", "% Passing Reading", "% Overall Passing Rate" ]]
df["Total Students"]=df["Total Students"].map("{:,}".format)
df["Total Budget"]=df["Total Budget"].map("${:,}".format)

df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
del school_data["School ID"]
school_data['Per Student Budget'] = school_data['budget']/school_data['size']
avg_math_read = student_data.groupby(['school_name'])['reading_score','math_score'].mean()
school_data = school_data.merge(avg_math_read, on='school_name', how="outer")

# students above 70 in math & readding
stud_above_70=school_data_complete.loc[school_data_complete["math_score"]>=70]
#count_stud_math=len(stud_above_70)
stud_above_70_read=school_data_complete.loc[school_data_complete["reading_score"]>=70]

#count the number of students >70  groupby school name in math & reading
pass_math_count = stud_above_70.groupby(["school_name"])['math_score'].count().reset_index()

pass_read_count = stud_above_70_read.groupby(["school_name"])['reading_score'].count().reset_index()

#merge the number of students data with school_data 

pass_count = pass_math_count.merge(pass_read_count, on='school_name', how="inner")
#print(pass_count)


#merge outer to include all the data
school_data = school_data.merge(pass_count, on="school_name", how='outer')

#calculate the percentage of math and reading students
school_data['per_math_stud']=(school_data['math_score_y']/school_data['size'])*100
school_data['per_read_stud']=(school_data['reading_score_y']/school_data['size'])*100

#delete of math_score_y & reading _score_y
del school_data["math_score_y"]
del school_data["reading_score_y"]

#overall passing rate
school_data['over_pass_rate']=school_data['per_math_stud'] + school_data['per_read_stud']/2
school_data_rename=school_data.rename(columns={"type":"School Type","size":"Total Students","budget":"Total School Budget","math_score_x" :"Average Math Score","reading_score_x":"Average Reading Score",
                                              "per_math_stud":"% Passing Math","per_read_stud":"% Passing Reading","over_pass_rate":"% Overall Passing Rate"})

school_data_rename.head()



,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,106.342132
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,106.358087
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,141.794435
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,107.184466
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,141.961853


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [6]:
#Top schools sort values by overall Passing rate
school_data_rename= school_data_rename.sort_values(["% Overall Passing Rate"],ascending=False)
school_data_rename.head()

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,142.653391
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,142.567568
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,142.137538
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,141.961853
14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,93.272171,97.308869,141.926606


## Bottom Performing Schools (By Passing Rate)

In [7]:
#Sort and display the five worst-performing schools
school_data_rename= school_data_rename.sort_values(["% Overall Passing Rate"],ascending=False)
school_data_rename.tail()


,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,107.184466
12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,66.057551,81.222432,106.668767
11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,106.476619
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,106.358087
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,106.342132


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [22]:
#By using loc we apply conditional statements on grade column then groupby school_name
math_grade_9th=school_data_complete.loc[school_data_complete['grade']=="9th"].groupby("school_name").mean()['math_score']
math_grade_10th=school_data_complete.loc[school_data_complete['grade']=="10th"].groupby("school_name").mean()['math_score']
math_grade_11th=school_data_complete.loc[school_data_complete['grade']=="11th"].groupby("school_name").mean()['math_score']
math_grade_12th=school_data_complete.loc[school_data_complete['grade']=="12th"].groupby("school_name").mean()['math_score']
#math_grade_9th
df_9=pd.DataFrame(data={"9th":math_grade_9th,"10th":math_grade_10th,"11th":math_grade_11th,"12th":math_grade_12th})
df_9.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [11]:
#By using loc we apply conditional statements on grade column then groupby school_name
Read_grade_9th=school_data_complete.loc[school_data_complete['grade']=="9th"].groupby("school_name").mean()['reading_score']
Read_grade_10th=school_data_complete.loc[school_data_complete['grade']=="10th"].groupby("school_name").mean()['reading_score']
Read_grade_11th=school_data_complete.loc[school_data_complete['grade']=="11th"].groupby("school_name").mean()['reading_score']
Read_grade_12th=school_data_complete.loc[school_data_complete['grade']=="12th"].groupby("school_name").mean()['reading_score']

In [23]:
df_read=pd.DataFrame(data={"9th":Read_grade_9th,"10th":Read_grade_10th,"11th":Read_grade_11th,"12th":Read_grade_12th})
df_read.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [15]:
school_data_rename['Spending Range']= pd.cut(school_data_rename['Per Student Budget'],spending_bins, labels=group_names)
#school_spending_rename=school_spending_bin.rename(columns={"Per Student Budget":"Spending Range"})
#school_spending_bin['Spending Range']=school_spending_bin
#scores_school_spending.head()
#school_data_rename

In [16]:
scores_school_spending=school_data_rename.groupby(['Spending Range'])['Average Reading Score','Average Math Score',
                                                                      '% Passing Math','% Passing Reading',
                                                                      '% Overall Passing Rate'].mean()
scores_school_spending

,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Range,,,,,
<$585,83.933814,83.455399,93.460096,96.610877,141.765534
$585-615,83.885211,83.599686,94.230858,95.900287,142.181001
$615-645,81.891436,79.079225,75.668212,86.106569,118.721496
$645-675,81.027843,76.997210,66.164813,81.133951,106.731788


## Scores by School Size

* Perform the same operations as above, based on school size.

In [17]:
#scores_school_size=school_data_rename.copy()

In [35]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [36]:
school_data_rename['School size']= pd.cut(school_data_rename['Total Students'],size_bins, labels=group_names)
#school_size_bin=pd.DataFrame(school_size_bin)
#school_data_rename['School size'] = school_size_bin
#school_data_rename.head()

In [34]:
scores_school_size=school_data_rename.groupby(["School size"])['Average Reading Score','Average Math Score',
                                                          '% Passing Math','% Passing Reading',
                                                           '% Overall Passing Rate'].mean()
#scores_school_spending=scores_school_spending.sort_values(["Spending Range"],ascending=True)
scores_school_size.head()

,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School size,,,,,
Small (<1000),83.929843,83.821598,93.550225,96.099437,141.599943
Medium (1000-2000),83.864438,83.374684,93.599695,96.790680,141.995035
Large (2000-5000),81.344493,77.746417,69.963361,82.766634,111.346678


## Scores by School Type

* Perform the same operations as above, based on school type.

In [37]:
score_school_type=school_data_rename.copy()
scores_school_type_df=score_school_type.groupby(['School Type'])['Average Reading Score','Average Math Score',
                                                             '% Passing Math','% Passing Reading',
                                                              '% Overall Passing Rate'].mean()
scores_school_type_df

,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.896421,83.473852,93.620830,96.586489,141.914075
District,80.966636,76.956733,66.548453,80.799062,106.947984


#         Analysis of Data
1.From the table and description of data we analyze that charters school are good at average scores and ovrerall Passing rate than the district schools.
2.Medium school size(<1000 - 2000) and Medium spending range of individual school($585-615) has more overall passing rate.
